<a href="https://colab.research.google.com/github/adryduty/computer-vision-cat-project/blob/main/Preprocessing_comp_vision_cat(11_June).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import io
import numpy as np    
from PIL import Image
import pickle
from google.colab import files, drive
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import time
import matplotlib.patches as patches
import tensorflow as tf
import pandas as pd
import os
import shutil
from math import floor
from sklearn.model_selection import train_test_split

# Here we are going to create some functions to take and save photos and to use them as input

### Almost all the lines of code there are here need to be run just one time (after that you just work with the pictures you saved).

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  '''
  This function creates a javascript function that takes a picture every 4000 milliseconds. 
  After that it displays the taken picture in the quality determined as argument, decode the picture in Base64 (binary) 
  and eventually returns the picture in binary.
  '''
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      
      await new Promise(r => setTimeout(r, 2000));

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  return binary

In [ ]:
def binary_to_numpy(binary):
  '''
  This function takes a picture coded in binary and transforms it as numpy array.
  '''
  img = Image.open(io.BytesIO(binary))
  # display(img)
  np_array = np.asarray(img)
  return np_array

* Assign an empty list to photo_array (it will take the numpy arrays of the pictures);
* Decide how many pictures to take and assign this number to num_of_pictures_to_take;
* Iterate num_of_pictures_to_take times and every time append the taken picture (in numpy array) in the photo_array.
* If the user doesn't have a webcam or if he doesn't grant the page permission to access it, return an error.

In [ ]:
photo_array = []

num_of_pictures_to_take = 15

for i in range(num_of_pictures_to_take):
  try:
    binary = take_photo()
    # img = Image.fromarray(binary_to_numpy(binary)).resize((416, 416))
    photo_array.append(binary_to_numpy(binary))
    
  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def show_image(np_array):
  '''
  This function takes a numpy array and displays the image relative to this numpy array.
  '''
  display(Image.fromarray(np_array))

In [ ]:
photo_array_2_5 = photo_array_2 + photo_array_3 + photo_array_4 + photo_array_5

#### See the images you have taken.

In [ ]:
for i in range(len(photo_array_2_5)):
  show_image(photo_array_2_5[i])
  print(i)

#### Choose the ones you want to keep and store them in photo_array_*_new

In [ ]:
photo_array_2_5_new = []

for i in range(len(photo_array_2_5)):
  if i not in [2, 13, 23, 25, 26, 27, 28, 34, 35, 36, 39, 42, 50, 52, 53, 61, 64, 73, 79]:
    photo_array_2_5_new.append(photo_array_2_5[i])

#### See the images you decided to keep

In [ ]:
for i in range(len(photo_array_2_5_new)):
  show_image(photo_array_2_5_new[i])
  print(i)

#### Save them in a folder you have already created and assign to each picture a different name (in this case img***.jpg)

In [ ]:
for i, image in enumerate(photo_array_2_5_new):
  j = 182
  img = Image.fromarray(image).resize((416, 416))
  img.save('/content/to_save_1/img'+ str(j+i) + '.jpg')

#### Create a zip of the colab folder where you stored the pictures 

In [ ]:
!zip -r /content/to_save_1.zip /content/to_save_1

#### Download the zip in the download repo of your local pc.

In [ ]:
files.download('/content/to_save_1.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Labelling (bounding box)
### The idea is to create a 20x20 grid on the image to choose the rectangle around Ghiri (you just need to choose the upper left square and the bottom right square).
#### Notice that the considered images are resized to 416x416

In [ ]:
def label_maker(upper_left, bottom_right, n_cells):
  '''
  CONSIDER THAT YOU HAVE A 20x20 MATRIX AND THAT THE PIXELS ARE NORMALIZED TO 1 (they go from 0 to 1)

  This function takes the upper left and the bottom right squares as input and returns:
  - 0 (indicating the class: here is just 0 because we have just Ghiri)
  - X_c (X coordinate of the center of the bounding box)
  - Y_C (Y coordinate of the center of the bounding box)
  - Width of the bounding box
  - Height of the buonding box
  '''
  
  if (bottom_right % n_cells) < (upper_left % n_cells):
    print("Wrong input")
    
  elif (upper_left // n_cells) > (bottom_right // n_cells):
    print("Wrong input")

  else:
    num_squares_height = 1 + (bottom_right - upper_left) // n_cells
    height = num_squares_height / n_cells

    num_squares_width = 1 + (bottom_right - upper_left) % n_cells
    width = num_squares_width / n_cells 
    x_a = (upper_left % n_cells + 0.5) / n_cells
    y_a = (upper_left // n_cells + 0.5) / n_cells

    x_b = (bottom_right % n_cells + 0.5) / n_cells
    y_b = (bottom_right // n_cells + 0.5) / n_cells

    x_c = (x_a + x_b)/2
    y_c = (y_a + y_b)/2

    return 0, x_c, y_c, width, height 


In [ ]:
# This is a 10x10 grid for just understanding how the above function works
np.arange(10)+np.arange(0,100,10)[:,np.newaxis]

In [ ]:
label_maker(15,43,10)

In [ ]:
def grid_image(google_path_image):
  '''
  This function takes as input the google path of the image you want to apply the grid onto.
  The output is the picture with the squares. 
  '''
  # Open image file
  img = Image.open(google_path_image)
  my_dpi=416.

  img = img.resize(size=(416, 416))

  # Set up figure
  fig=plt.figure(figsize=(float(img.size[0])/my_dpi,float(img.size[1])/my_dpi),dpi=my_dpi)
  ax=fig.add_subplot(111)

  # Remove whitespace from around the image
  fig.subplots_adjust(left=0,right=1,bottom=0,top=1)

  # Set the gridding interval: here we use the major tick interval
  myInterval= 20.8
  loc = plticker.MultipleLocator(base=myInterval)
  ax.xaxis.set_major_locator(loc)
  ax.yaxis.set_major_locator(loc)

  ax.tick_params(axis='x', colors=(0,0,0,0))
  ax.tick_params(axis='y', colors=(0,0,0,0))

  # Remove the borders 
  ax.spines['top'].set_visible(False) 
  ax.spines['right'].set_visible(False) 
  ax.spines['left'].set_visible(False)
  ax.spines['bottom'].set_visible(False)

  # Add the grid
  ax.grid(which='major', axis='both', linestyle='-', linewidth=0.1)

  # Add the image
  ax.imshow(img)

  # Find number of gridsquares in x and y direction
  nx=abs(int(float(ax.get_xlim()[1]-ax.get_xlim()[0])/float(myInterval)))
  ny=abs(int(float(ax.get_ylim()[1]-ax.get_ylim()[0])/float(myInterval)))

  # Add some labels to the gridsquares
  for j in range(ny):
      y=myInterval/2+j * myInterval
      for i in range(nx):
          x=myInterval/2.+float(i) * myInterval
          ax.text(x,y,'{:d}'.format(i+j*nx),color='w',ha='center',va='center', fontsize=1.4)

  plt.show()



In [ ]:
# THIS IS JUST A CHUNK FOR UNDERSTANDING WHAT'S NEXT (DRAWING A RECTANGLE AROUND GHIRI)

im = Image.open('/content/27GHIRI.jpg')
im = im.resize(size=(416,416))

# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(im)

# Create a Rectangle patch
rect = patches.Rectangle((0.55*416, 0.75*416), 0.25*416, 0.2*416, linewidth=1, edgecolor='r', facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

plt.show()

In [ ]:
def rectangle_coordinates(x_c, y_c, width, height):
  '''
  This function takes the coordinates of the rectangle's center and the width and the height of a
  rectangle and returns the coordinates of the rectangle's bottom left angle and the width and the height
  multiplied by 416. 
  
  These coordinates will be important to make the rectangle (see the rectangle_maker function)
  '''

  x_bottom_left_angle = (x_c - width/2)*416

  y_bottom_left_angle = (y_c - height/2)*416

  width = width * 416

  height = height * 416

  return x_bottom_left_angle, y_bottom_left_angle, width, height



In [ ]:
def rectangle_maker(google_path_image, x_c, y_c, width, height):

  '''
  This function takes a picture saved in colab, the coordinates of the centers of a rectangle, its width and its height.
  It returns the image with the rectangle around Ghiri and also saved it on colab
  '''

  im = Image.open(google_path_image)
  im = im.resize(size=(416,416))

  # Create figure and axes
  fig, ax = plt.subplots()
  
  x_left_angle, y_left_angle, denorm_width, denorm_height = rectangle_coordinates(x_c, y_c, width, height)
  
  # Create a Rectangle patch
  rect = patches.Rectangle((x_left_angle, y_left_angle), denorm_width, denorm_height, linewidth=1, edgecolor='r', facecolor='none')

  # Add the patch to the Axes
  ax.add_patch(rect)

  ax.tick_params(axis='x', colors=(0,0,0,0))
  ax.tick_params(axis='y', colors=(0,0,0,0))

  # Remove the borders 
  ax.spines['top'].set_visible(False) 
  ax.spines['right'].set_visible(False) 
  ax.spines['left'].set_visible(False)
  ax.spines['bottom'].set_visible(False)

  # Display the image
  ax.imshow(im)

  plt.savefig('/' + google_path_image.split("/")[1]+'/with_rectangle_'+google_path_image.split("/")[2])


In [ ]:
rectangle_maker('/content/27GHIRI.jpg', 0.6499999999999999, 0.8500000000000001, 0.25, 0.2)

### Let's create a list with all the names of the pictures saved on colab (by hand). It's necessary for the next step.

In [ ]:
path_list = []
numbers = ['00','01', '02', '03', '04','05','06','07','08','09',10,11,12,13,14,15,16,17,20,21,22,23,
             24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,'010','011','012','013','014','015','016','017',
             210,211,212,213,214,310,311,312,313]
for number in numbers:
  path_list.append("/content/" + str(number) + "GHIRI.jpg")

In [ ]:
path_list = []
numbers = range(163,184)
for number in numbers:
  path_list.append("/content/img" + str(number) + ".jpg")

### The following chunk allows the user to insert the upper left and the bottom right squares to make the rectangles and so FOR LABELING. The labels will be stored into the **label_vector** list.

#### Each label will be an array **['google_path', 'label', 'x_c', 'y_c', 'width', 'height']**

In [ ]:
label_vector = []

for path in path_list:

  grid_image(path)

  time.sleep(3)

  upper_left = int(input("insert upper left square: "))

  bottom_right = int(input("insert bottom right square: "))
  
  label, x_c, y_c, width, height = label_maker(upper_left, bottom_right, 20)

  label_vector.append([path, label, x_c, y_c, width, height])

  rectangle_maker(path, x_c, y_c, width, height)

## Transform the label_vecor into a dataframe and store it as csv 

In [ ]:
label_dataframe = pd.DataFrame(label_vector, columns = ['google_path', 'label', 'x_c', 'y_c', 'width', 'height'])
label_dataframe.to_csv('/content/label_dataframe.csv', index = False )

### The code above was just for 55 images. In the next one I concatenate the csv with 55 images with another csv with 99 images.

In [ ]:
label_55 = pd.read_csv('/content/label_dataframe.csv')
label_99 = pd.read_csv('/content/label_dataframeTot.csv')

label_154 = pd.concat([label_55, label_99])

In [ ]:
label_154 = pd.DataFrame(label_154, columns = ['google_path', 'label', 'x_c', 'y_c', 'width', 'height'])
label_154.to_csv('/content/label_154.csv', index = False)

### The code above was just for 154 images. In the next one I concatenate the csv with 154 images with another csv with 21 images.

In [ ]:
label_154_183 = pd.DataFrame(label_vector, columns = ['google_path', 'label', 'x_c', 'y_c', 'width', 'height']) 

In [ ]:
label_154 = pd.read_csv('/content/label_154.csv')
label_183 = pd.concat([label_154, label_154_183])

In [ ]:
label_183.to_csv('/content/label_183.csv', index = False)

### The code above was just for 183 images. In the next one I concatenate the csv with 183 images with other two csv with a total of 59 images.

In [ ]:
label_183 = pd.read_csv('/content/label_183.csv')

In [ ]:
label_184_212 = pd.read_csv('/content/label_dataframe_184_212.csv')
label_213_242 = pd.read_csv('/content/label_dataframe_213_242.csv')

In [ ]:
label_242 = pd.concat([label_183, label_184_212, label_213_242])

In [ ]:
label_242.shape

(234, 6)

In [ ]:
label_242.to_csv('/content/label_242.csv', index = False)